In [ ]:
import numpy as np
import time
from math import log

import torch
import torch.optim as optim
from torch_geometric.nn import MessagePassing

import import_ipynb
from constants import *
from utils import *

In [ ]:
def DiffIM(adj_list, seed_idx, prob, del_edge_num, model_name=None, gnn_latent_dim=[128,128,128,128,128,128], gpu_num='cpu', **kwargs):
    time_start = time.time()
    time_alg = []
    
    device = set_gpu(gpu_num)
    model = load_model(model_name, device, gnn_latent_dim=gnn_latent_dim)

    # convert adj_list to Data
    data = adj2Data(adj_list, seed_idx)
    data = data.to(device)
    edge_index = data.edge_index
    edge_attr = data.edge_attr
    
    mask = []
    for _ in range(del_edge_num):
        pred_min = float('inf')
        idx_min = None
        for i in range(len(edge_attr)):
            u = edge_index[0,i]
            v = edge_index[1,i]
            p = edge_attr[i][0].item()
            if p==0: continue
            edge_attr[i][0] = 0
            pred = model(data)[0].item()
            edge_attr[i][0] = p
            if pred<pred_min: pred_min = pred; idx_min=(u,v,i,p)
        mask.append([idx_min[0],idx_min[1],idx_min[3]])
        edge_attr[idx_min[2]]=0

        time_alg.append(time.time()-time_start)
        
    return mask, time_alg

In [ ]:
from math import log

def DiffIMp(adj_list, seed_idx, prob, del_edge_num, model_name=None, gnn_latent_dim=[128,128,128,128,128,128], gpu_num='cpu', reset=False, **kwargs):
    time_alg = -time.time()

    # get parameters
    epochs = kwargs.get('epochs',100)
    lr = kwargs.get('lr',1.0)
    lr_gamma = kwargs.get('lr_gamma',1.0)
    size_coeff = kwargs.get('size_coeff',0.1)
    ent_coeff = kwargs.get('ent_coeff',1)
    log_filename = kwargs.get('log_filename',None)
    eps = 1e-15
    
    # load model
    device = set_gpu(gpu_num)
    model = load_model(model_name, device, gnn_latent_dim=gnn_latent_dim)

    # load data
    edges = adj2edges(adj_list)
    data = adj2Data(adj_list, seed_idx, prob=prob)
    data = data.to(device)
    x = data.x
    edge_index = data.edge_index
    edge_attr = data.edge_attr
    target=torch.unsqueeze(torch.sum(data.y,dim=0),0)
    m = len(edges)
    with torch.no_grad(): infl_orig = sum(model(x, edge_index, edge_attr=edge_attr)).item()
    seed_size = sum(x).item()

    # set edge mask
    tmp = 1-(del_edge_num/m)
    tmp = log(tmp)-log(1-tmp)
    edge_mask = torch.nn.Parameter(torch.ones((m,1), requires_grad=True, device=device)*tmp)

    # optimizing setup
    def _loss(y_hat, y, epoch):
        pred = y_hat[0]
        mask = edge_mask.sigmoid()
        size = m-mask.sum()
        for j in used_idx: size += mask[j][0]
        ent = (-mask*torch.log(mask+eps)-(1-mask)*torch.log(1-mask+eps)).mean()
        loss = -(infl_orig-pred)/(infl_orig-seed_size) + size_coeff*(size-del_edge_num)**2 + ent_coeff*ent
        return loss, pred, size, ent
    parameters = [edge_mask]
    optimizer = torch.optim.Adam(parameters, lr=lr)
    #scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: lr_gamma)

    # optimize
    record = []
    ret = []
    used_idx = []
    for _ in range(del_edge_num):
        for i in range(epochs):
            optimizer.zero_grad()
            edge_attr_with_mask = edge_attr*(edge_mask.sigmoid())
            y_hat, y = model(x, edge_index, edge_attr=edge_attr_with_mask), target
            loss, pred, size, ent = _loss(y_hat, y, i)
            record.append((i,loss.item(),pred.item(),size.item(),ent.item()))
            #print(i,loss.item(),pred.item(),size.item(),ent.item())
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(parameters, 0.01)
            optimizer.step()
            #scheduler.step()
        
        with torch.no_grad():
            max_val = torch.max(edge_mask)
            for j in used_idx: edge_mask[j,0] = max_val
            min_idx = torch.argmin(edge_mask)
            edge_attr[min_idx][0] = 0
        used_idx.append(min_idx)
        ret.append(edges[min_idx])

        if reset:
            with torch.no_grad(): edge_mask.fill_(tmp)
            optimizer = torch.optim.Adam(parameters, lr=lr)
    
    # save record
    if log_filename :
        log_dir, log_num = log_filename
        with open(log_dir+f'/train_{log_num}.txt','w') as f:
            f.write('seed infl_orig\n')
            f.write(f'{seed_size} {infl_orig}\n')
            f.write('epoch pred size ent loss\n')
            for epoch, loss, pred, size, ent in record: f.write(f'{epoch} {pred} {size} {ent} {loss}\n')
        with open(log_dir+f'/realmask_{log_num}.npy', 'wb') as f: np.save(f, edge_mask)

    time_alg += time.time()
    return ret, time_alg

In [ ]:
def DiffIMpp(adj_list, seed_idx, prob, del_edge_num, model_name=None, gnn_latent_dim=[128,128,128,128,128,128], gpu_num='cpu', **kwargs):
    time_start = time.time()
    time_alg = []

    # load model
    device = set_gpu(gpu_num)
    model = load_model(model_name, device, gnn_latent_dim=gnn_latent_dim)

    # load data
    edges = adj2edges(adj_list)
    data = adj2Data(adj_list, seed_idx, prob=prob)
    data = data.to(device)
    x = data.x
    edge_index = data.edge_index
    edge_attr = data.edge_attr
    m = len(edges)

    # select edges
    edge_attr.requires_grad = True
    ret = []
    for _ in range(del_edge_num):
        pred = model(x, edge_index, edge_attr=edge_attr)
        grad = torch.autograd.grad(pred, edge_attr)[0]
        with torch.no_grad():
            grad = grad*edge_attr
            max_idx = torch.argmax(grad)
            edge_attr[max_idx][0]=0
        ret.append(edges[max_idx])
        time_alg.append(time.time()-time_start)

    return ret, time_alg